In [ ]:
import numpy as np
import pandas as pd
import param
import panel as pn
pn.extension()

In [ ]:
import holoviews as hv
import hvplot.pandas

In [ ]:
df_cap = pd.read_csv('../results/capacity.csv')
df_fuel = pd.read_csv('../results/fuel_cons.csv')
df_ELsup = pd.read_csv('../results/Elsup.csv')
df_emissions = pd.read_csv('../results/emissions.csv')
df_trade = pd.read_csv('../results/trade.csv')
df_investments = pd.read_csv('../results/investments.csv')

In [ ]:
df_investments['value'] = np.divide(df_investments['value'],1000)

In [ ]:
names = {'bah':'Bahrain',
         'kuw':'Kuwait',
         'ksa':'Saudi Arabia',
         'omn':'Oman',
         'qat':'Qatar',
         'uae':'UAE'}

df_cap = df_cap.replace(names)
df_fuel = df_fuel.replace(names)
df_ELsup = df_ELsup.replace(names)
df_emissions = df_emissions.replace(names)
df_trade = df_trade.replace(names)
df_investments = df_investments.replace(names)

In [ ]:
countries = ['Bahrain','Kuwait','Oman','Qatar','Saudi Arabia','UAE']
fuels = ['HFO','diesel','NG','Arablight']
scenarios = ['A','B','C','D','E','F','G','H']

In [ ]:
df_trade.head(2)

In [ ]:
class GENERAL(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    scenario_selector = param.ObjectSelector(default="G", objects=scenarios)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    #show_legend = param.Boolean(False, doc="Show the legend")

# CAPACITY ----------------------------------------------------------------   


    
#    def plot_cap(self):
#        country = self.country_selector
#        _df  = self.get_data_cap(country)
#        mask = (_df['scenario']==self.scenario_selector)
#        data = _df.loc[mask].reset_index(drop=True)      
#        
#        title = country + ' ' + self.scenario_selector
#        
#        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()
#
#        plot = data.hvplot.bar(x='trun',
#                               y=ELp_list,
#                               value_label='GW',
#                               title=title,
#                               alpha=0.4,
#                               stacked=True,
#                               width=300,
#                               legend=self.show_legend,
#                               shared_axes=False,
#                               rot=90)
#        return plot

# FUEL ----------------------------------------------------------------    
    
    #def get_data_fuel(self,country):
    #    start_date = self.year_range[0]
    #    end_date = self.year_range[1]
    #    #country = self.country_selector
    #    
    #    mask = (df_fuel['c'] == country) & (df_fuel['trun'] >= start_date) & (df_fuel['trun'] <= end_date)
    #    
    #    _df = df_fuel.loc[mask]
    #    
    #    #df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
    #    #df_pivot = df_pivot.reset_index(drop=False)
    #    
    #    return _df

    #def plot_fuel(self):     
    #    country = self.country_selector
    #    _df  = self.get_data_fuel(country)
    #    mask = (_df['scenario']==self.scenario_selector)               # come back to this
    #    data = _df.loc[mask].reset_index(drop=True)      
    #    
    #    title = self.country_selector + ' ' + self.scenario_selector
    #    
    #    fuel_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()
    #    
    #    plot = data.hvplot.bar(x='trun',
    #                           y=fuel_list,
    #                           value_label='GW',
    #                           title=title,
    #                           alpha=0.4,
    #                           stacked=True,
    #                           width=300,
    #                           legend=self.show_legend,
    #                           shared_axes=False,
    #                           rot=90)
    #    return plot
    

# ELECTRICITY SUPPLY ---------------------------------------------------------------   

    #def get_data_sup(self,country):
    #    start_date = self.year_range[0]
    #    end_date = self.year_range[1]
    #    #country = self.country_selector
    #    
    #    mask =  (df_ELsup['c'] == country) & (df_ELsup['trun'] >= start_date) & (df_ELsup['trun'] <= end_date)
    #    
    #    _df = df_ELsup.loc[mask]
    #    
    #    df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
    #    df_pivot = df_pivot.reset_index(drop=False)
    #    
    #    return df_pivot
    
#    def plot_sup(self):
#        country = self.country_selector
#        _df  = self.get_data_sup(country)
#        mask = (_df['scenario']==self.scenario_selector)
#        data = _df.loc[mask].reset_index(drop=True)      
#        
#        title = self.country_selector + ' ' + self.scenario_selector
#        
#        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()
#
#        plot = data.hvplot.bar(x='trun',
#                               y=ELp_list,
#                               value_label='GW',
#                               title=title,
#                               alpha=0.4,
#                               stacked=True,
#                               width=300,
#                               legend=self.show_legend,
#                               shared_axes=False,
#                               rot=90)
#        return plot

# EMISSIONS ---------------------------------------------------------------   

    #def get_data_emissions(self,country):
    #    start_date = self.year_range[0]
    #    end_date = self.year_range[1]
    #    
    #    mask = (df_emissions['c'] == country) & (df_emissions['trun'] >= start_date) & (df_emissions['trun'] <= end_date)
    #    _df = df_emissions.loc[mask]
    #    
    #    #df_pivot = _df.pivot_table(index=['trun','scenario'],columns='c',values='value',fill_value=0)
    #    #df_pivot = df_pivot.reset_index(drop=False)
    #    
    #    return _df
    
#    def plot_emissions(self):
#        country = self.country_selector
#        _df  = self.get_data_emissions(country)
#        mask = (_df['scenario']==self.scenario_selector)
#        data = _df.loc[mask].reset_index(drop=True)   
#        
#        title = 'Emissions for ' + country + ' for ' + self.scenario_selector
#        
#        #c_list = data.drop(['trun','scenario'],axis=1).columns.values.tolist()
#
#        plot = data.hvplot(
#            x='trun',
#            value_label='million tons',
#            title=title,
#            alpha=0.4,
#            width=300,
#            legend=self.show_legend,
#            shared_axes=False,
#            rot=90)* data.hvplot.scatter(
#            x='trun',
#            title=title,
#            line_color='k',
#            shared_axes=True)
#        return plot

    

In [ ]:
class EMISSIONS(param.Parameterized):
    
# EMISSIONS SUBPLOTS
# https://hvplot.holoviz.org/user_guide/Subplots.html
    
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))

    def get_data_emissions(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        country = self.country_selector
        
        mask = (df_emissions['c'] == country) & (df_emissions['trun'] >= start_date) & (df_emissions['trun'] <= end_date)
        _df = df_emissions.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['trun','scenario'],columns='c',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df
    
    def plot_emissions_grid(self):
        country = self.country_selector
        _df  = self.get_data_emissions()
        
        title = 'Emissions for ' + country
        
        plot = (_df.hvplot(
            x='trun',
            y='value',
            by='scenario',
            subplots=True,
            value_label='million tons',
            alpha=0.7,
            width=300,
            shared_axes=True,
            rot=90) * _df.hvplot.scatter(
            x='trun',
            y='value',
            by='scenario',
            subplots=True,
            title=title,
            line_color='k',
            shared_axes=True)).cols(4)
        return plot

In [ ]:
em = EMISSIONS()
em.plot_emissions_grid()

In [ ]:
class FUEL(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data_fuel(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        country = self.country_selector
        
        mask = (df_fuel['c'] == country) & (df_fuel['trun'] >= start_date) & (df_fuel['trun'] <= end_date)
        
        _df = df_fuel.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df
    
    def plot_fuel_grid(self,scenario):    

        _df  = self.get_data_fuel()
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        mask = (df_pivot['scenario']==scenario)               # come back to this
        data = df_pivot.loc[mask].reset_index(drop=True)      
        
        title = self.country_selector + ' Scenario ' + scenario
        
        fuel_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(
            x='trun',
            y=fuel_list,
            #by='scenario',
            stacked=True,
            subplots=False,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)
        return plot

In [ ]:
fuel = FUEL()
fuelA = fuel.plot_fuel_grid('A')
fuelB = fuel.plot_fuel_grid('B')
fuelC = fuel.plot_fuel_grid('C')
fuelD = fuel.plot_fuel_grid('D')
fuelE = fuel.plot_fuel_grid('E')
fuelF = fuel.plot_fuel_grid('F')
fuelG = fuel.plot_fuel_grid('G')
fuelH = fuel.plot_fuel_grid('H')
layout_fuel = hv.Layout(fuelA+fuelB+fuelC+fuelD+fuelE+fuelF+fuelG+fuelH).cols(4)
layout_fuel

In [ ]:
class ELSUP(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data_sup(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        country = self.country_selector
        
        mask =  (df_ELsup['c'] == country) & (df_ELsup['trun'] >= start_date) & (df_ELsup['trun'] <= end_date)
        
        _df = df_ELsup.loc[mask]
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot


    
    def plot_sup(self,scenario):
        country = self.country_selector
        _df  = self.get_data_sup()
        
        #df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        mask = (_df['scenario']==scenario)
        data = _df.loc[mask].reset_index(drop=True)      

        title = self.country_selector + ' for Scenario ' + scenario
        
        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(
            x='trun',
            y=ELp_list,
            value_label='GW',
            title=title,
            alpha=0.4,
            stacked=True,
            width=300,
            shared_axes=False,
            legend=self.show_legend,
            rot=90)
        return plot

    #def plot_sup2(self):
    #    country = self.country_selector
    #    _df  = self.get_data_sup(country)
    #    mask = (_df['scenario']==self.scenario_selector)
    #    data = _df.loc[mask].reset_index(drop=True)      
    #    
    #    title = self.country_selector + ' ' + self.scenario_selector
    #    
    #    ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()
    #    
    #    plot = data.hvplot.bar(x='trun',
    #                           y=ELp_list,
    #                           value_label='GW',
    #                           title=title,
    #                           alpha=0.4,
    #                           stacked=True,
    #                           width=300,
    #                           shared_axes=False,
    #                           rot=90)
    #    return plot

In [ ]:
sup = ELSUP()
supA = sup.plot_sup('A')
supB = sup.plot_sup('B')
supC = sup.plot_sup('C')
supD = sup.plot_sup('D')
supE = sup.plot_sup('E')
supF = sup.plot_sup('F')
supG = sup.plot_sup('G')
supH = sup.plot_sup('H')
layout_sup = hv.Layout(supA+supB+supC+supD+supE+supF+supG+supH).cols(4)
layout_sup

In [ ]:
class CAPACITY(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    #scenario_selector = param.ObjectSelector(default="G", objects=scenarios)
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data_cap(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        country = self.country_selector
        
        mask =  (df_cap['c'] == country) & (df_cap['trun'] >= start_date) & (df_cap['trun'] <= end_date)
        
        _df = df_cap.loc[mask]
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
   
    def plot_cap(self,scenario):
        country = self.country_selector
        #scenario = self.scenario_selector
        _df  = self.get_data_cap()
        mask = (_df['scenario']==scenario)
        data = _df.loc[mask].reset_index(drop=True)      
        
        title = country + ' for Scenario ' + scenario
        
        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(x='trun',
                               y=ELp_list,
                               value_label='GW',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=300,
                               shared_axes=False,
                               legend=self.show_legend,
                               rot=90)
        return plot

In [ ]:
cap = CAPACITY()
capA = cap.plot_cap('A')
capB = cap.plot_cap('B')
capC = cap.plot_cap('C')
capD = cap.plot_cap('D')
capE = cap.plot_cap('E')
capF = cap.plot_cap('F')
capG = cap.plot_cap('G')
capH = cap.plot_cap('H')
layout_cap = hv.Layout(capA+capB+capC+capD+capE+capF+capG+capH).cols(4)
layout_cap

In [ ]:
#dashboard = pn.Tabs(('At a glance',
#                     pn.Column(
#                     pn.Row(g.param),
#                     pn.Row(g.plot_cap,g.plot_fuel),
#                     pn.Row(g.plot_el,g.plot_emissions))
#))

dashboard = pn.Tabs(('Capacity',
             pn.Column(pn.Row('## Capacities'),
             pn.Row(cap.param),
             pn.Row(layout_cap))                   
            ))

dashboard.append(('Fuel',
             pn.Column(pn.Row('## Fuel consumption'),
                       pn.Row(fuel.param),
                       pn.Row(layout_fuel))        
            ))

dashboard.append(('Electricity',
             pn.Column(
             pn.Row('## Electricity supply'),
             pn.Row(sup.param),
             pn.Row(layout_sup))                      
            ))

dashboard.append(('Emissions',
             pn.Column(
             pn.Row('## CO2 emissions'),
             pn.Row(em.param),
             pn.Row(em.plot_emissions_grid()))                     
            ))

#dashboard.append(('Exchange',
#             pn.Column(
#                 pn.Row('## Electricity exchange'),
#                 pn.Row(ex.param),
#                 pn.Row(ex.plot_C,ex.table_C),
#                 pn.Row(ex.plot_D,ex.table_D),
#                 pn.Row(ex.plot_G,ex.table_G),
#                 pn.Row(ex.plot_H,ex.table_H))
#             ))
#
#dashboard.append(('Investment',
#                 pn.Column(
#                     pn.Row(inv.param),
#                     pn.Row(inv.plot_data))
#                 ))
#
#dashboard.append(('Side by side',
#                 pn.Column(pn.Row(c.param.country_selector1,c.param.scenario_selector,c.param.show_legend),
#                 pn.Row(c.param.country_selector2,c.param.year_range),
#                 pn.Row('## Side by side country comparisons'),
#                 pn.Row(c.cap_plot1,c.cap_plot2),
#                 pn.Row(c.fuel_plot1,c.fuel_plot2),
#                 pn.Row(c.sup_plot1,c.sup_plot2))
#                 ))

In [ ]:
dashboard

In [ ]:
I = INVESTMENTS()
I.plot_data()

In [ ]:
class EXCHANGE(GENERAL):
    def get_data_exchange(self):
        _df = df_trade
        _df = df_trade.groupby(['scenario','c','cc']).sum().drop('trun',axis=1)
        _df = _df.reset_index().drop('scenario',axis=1)
        display(_df)
        plot = hv.Chord(_df)
        return plot

In [ ]:
J = EXCHANGE()
J.get_data_exchange()

In [ ]:
#! panel serve --show --port 5000 dashboard.ipynb